In [31]:
import pandas as pd

In [32]:
df = pd.read_csv('cops.cleaned.translated.csv')

In [33]:
df.tail(100)

,Group_Type,Delegation,Honorific,Person_Name,Job_Title,Division,Affiliation,Virtual,Overflow,Year,Meeting,Location,Female,IGO,NGO,Observer,Party,IO,Delegation_COW,Delegation_ISO
367641,Non-governmental organizations,Whitworth University,Ms,Melaina Kloberdanz,Student - Whitworth University,Political Science,Whitworth University,1,0,2024,COP 29,Baku,1.0,0,1,0,0,0.0,NaN,NaN
367642,Non-governmental organizations,Whitworth University,Ms,Grace Hansen,Student - Whitworth University,Political Science,Whitworth University,1,0,2024,COP 29,Baku,1.0,0,1,0,0,0.0,NaN,NaN
367643,Non-governmental organizations,Whitworth University,Ms,Ingrid Burchill,Student - Whitworth University,Political Science,Whitworth University,1,0,2024,COP 29,Baku,1.0,0,1,0,0,0.0,NaN,NaN
367644,Non-governmental organizations,Whitworth University,Mr,Jack Odone,Student - Whitworth University,Political Science,Whitworth University,1,0,2024,COP 29,Baku,0.0,0,1,0,0,0.0,NaN,NaN
367645,Non-governmental organizations,Whitworth University,Ms,Isabella Martinez,Student - Whitworth University,Political Science,Whitworth University,1,0,2024,COP 29,Baku,1.0,0,1,0,0,0.0,NaN,NaN
367646,Non-governmental organizations,Women Deliver,Ms,Heritage Foluwakemi Sanmi-Lawal,Executive Director,Advocacy,Her Dream Initiative,1,0,2024,COP 29,Baku,1.0,0,1,0,0,0.0,NaN,NaN
367647,Non-governmental organizations,Women Deliver,Ms,Leila Rashid Mgalla,Emerging Leader,NaN,Dada Kazini Foundation,1,0,2024,COP 29,Baku,1.0,0,1,0,0,0.0,NaN,NaN
367648,Non-governmental organizations,Women Deliver,Ms,Nagla Abbas Mdungi,Emerging Leader,NaN,Assalam Community Foundation,1,0,2024,COP 29,Baku,1.0,0,1,0,0,0.0,NaN,NaN
367649,Non-governmental organizations,Women Engage for A Common Future - Internation...,Ms,Annabel Lucy Kennedy,Programm manager,Climate & gender projects,Women Engage for a Common Future - Internation...,1,0,2024,COP 29,Baku,1.0,0,1,0,0,0.0,NaN,NaN
367650,Non-governmental organizations,Women In Renewable Energy,Ms,Dana Darwish,Chapter Leader,WiRE UAE,Women in Renewable Energy (WiRE),1,0,2024,COP 29,Baku,1.0,0,1,0,0,0.0,NaN,NaN


In [34]:
df['Division'].value_counts()

Division
Climate Change                                                              1450
Not specified                                                               1390
Management                                                                  1302
Environment                                                                 1124
Communications                                                              1043
                                                                            ... 
Office of Atmospheric Programs Office of Air and Radiation United States       1
Oro Governors Office                                                           1
CfRN Secretariat                                                               1
PNG                                                                            1
The Climate Society                                                            1
Name: count, Length: 64818, dtype: int64

In [35]:
df['Group_Type'].value_counts()

Group_Type
Parties                                           197619
Non-governmental organizations                    140903
Intergovernmental organizations                    12697
United Nations Secretariat units and bodies         8626
Specialized agencies and related organizations      7339
Observer States                                      557
Name: count, dtype: int64

In [36]:
pd.options.display.max_rows = 500

In [37]:
df['Job_Title'].value_counts().head(500)

Job_Title
Director                                        14128
Executive Director                               3855
Advisor                                          3218
President                                        3194
Deputy Director                                  3141
Manager                                          2747
Expert                                           2589
Student                                          2513
Minister                                         2494
Researcher                                       2234
CEO                                              2211
Head                                             2085
Professor                                        1992
Consultant                                       1990
Member                                           1977
Coordinator                                      1926
Ambassador                                       1772
Project Manager                                  1531
Director General  

In [38]:
df['Job_Title'].str.contains('Journalist', case=False).value_counts()

Job_Title
False    280232
True        994
Name: count, dtype: int64

In [39]:
from transformers import pipeline

pipe = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

Device set to use mps:0


In [40]:
def get_category(Job_Title):
    categories = ["Leadership & Executive Roles", "Directors & Department Heads", "Managerial Roles", "Advisory Roles", "Diplomatic & Government Roles",
                 "Political Roles","Coordination Roles","Expert & Specialist Roles","Academic & Research Roles","Student Roles","Professional Roles","Media & Journalism Roles",
                 "Communications & PR Roles","Administrative & Support Roles","Consulting & Analysis Roles","Membership & Representative Roles","Sustainability & Environmental Roles",
                 "Translator & Interpreter Roles","Other/Miscellaneous"]
    
    result = pipe(Job_Title, categories)
    return result['labels'][0], result['scores'][0]

get_category("Student - Whitworth University")

('Student Roles', 0.30620062351226807)

In [41]:
import csv

In [42]:
from tqdm import tqdm

In [43]:
tqdm.pandas()

In [45]:
# Ensure df exists
df_subset = df['Job_Title'].value_counts().reset_index()
df_subset.columns = ['Job_Title', 'Count']  # Rename columns properly

output_file = "categorized_results_job_titles.csv"

# Write headers to the CSV file
with open(output_file, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["Job Title", "Count", "Category", "Confidence Score"])  # Ensure correct columns

chunk_size = 10_000
for i in range(0, len(df_subset), chunk_size):
    df_chunk = df_subset.iloc[i:i + chunk_size].copy()  # Copy to avoid Pandas warning

    # Apply categorization function
    df_chunk[['Category', 'Confidence Score']] = df_chunk['Job_Title'].progress_apply(
        lambda x: pd.Series(get_category(x))
    )

    # Append results to CSV
    df_chunk.to_csv(output_file, mode='a', index=False, header=False)  

print("Processing complete. Results saved to", output_file)


100%|█████████████████████████████████████| 6184/6184 [2:09:34<00:00,  1.26s/it]

Processing complete. Results saved to categorized_results_job_titles.csv


In [46]:
df['Delegation_ISO'].value_counts().head(500)

Delegation_ISO
BRA    7971
ARE    6418
CHN    4323
JPN    4248
IDN    4015
FRA    3961
CAN    3942
MAR    3885
AZE    3781
USA    3698
TUR    3576
KOR    3201
DEU    3066
NGA    2900
GBR    2682
RUS    2663
KEN    2614
ITA    2367
DNK    2322
IND    2235
COD    2199
EUE    2111
ZAF    2110
SEN    2095
GHA    1824
UGA    1778
AUS    1763
POL    1754
SWE    1666
THA    1624
NOR    1545
TZA    1544
ESP    1521
BEL    1507
MYS    1506
BGD    1497
SAU    1485
CIV    1455
FIN    1453
PHL    1441
QAT    1372
ZWE    1367
EGY    1356
ISR    1335
CHL    1310
PER    1302
BFA    1291
NLD    1271
IRQ    1241
SGP    1240
COG    1230
KAZ    1199
ETH    1161
ZMB    1142
PRT    1141
VNM    1131
BEN    1117
MLI    1099
PAK    1092
MEX    1068
COL    1063
GIN    1012
MWI    1006
TCD    1005
AUT     996
TUN     996
KWT     984
AGO     981
TGO     974
MRT     957
GRC     945
ARG     930
NPL     922
JOR     916
UKR     900
SDN     895
IRL     884
NAM     865
MDG     854
PNG     830
NER     828
OMN     821
M